In [4]:
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.chemistry.aqua_extensions.components.variational_forms import UCCSD
from qiskit.chemistry.core import Hamiltonian, TransformationType, QubitMappingType
from qiskit.chemistry import FermionicOperator
import numpy as np
import os
from hucc import hUCC
import logging
import copy
from qiskit import QuantumRegister
from qiskit import Aer, IBMQ
from qiskit.chemistry.aqua_extensions.components.initial_states import HartreeFock
from qiskit.aqua.operators import WeightedPauliOperator
from qiskit import execute
from qiskit.aqua import QuantumInstance
from qiskit.chemistry.aqua_extensions.algorithms.q_equation_of_motion.q_equation_of_motion import QEquationOfMotion 
from qiskit.providers.aer import noise
import time


logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

two_qubit_reduction = False
qubit_mapping = 'jordan_wigner'

qasm_simulator = Aer.get_backend('qasm_simulator')
sv_simulator = Aer.get_backend('statevector_simulator')
IBMQ.save_account('c0a92cde985c4704aa127f053af0c0d3f351de572ac21dcdc37670c35b4f568a975128b54dbe3703c9b4a81cb3de1493b171a061f0f958fed02b16fb1d38cb3c', overwrite=True)
IBMQ.load_accounts()
provider_zrl = IBMQ.get_provider(hub='ibm-q-internal', group='zrl', project='main') 


qasm = True

if qasm:
    device = provider_zrl.get_backend('ibmq_poughkeepsie')
    properties = device.properties()
    coupling_map = device.configuration().coupling_map
    noise_model = noise.device.basic_device_noise_model(properties)
    basis_gates = noise_model.basis_gates
    #noise_model = None
    #basis_gates = None
    shots = 100000
    qubit_mapper = [2,1,0,5,6,7]
    quantum_instance = QuantumInstance(qasm_simulator, shots=shots, basis_gates=basis_gates, coupling_map=coupling_map, 
        initial_layout=qubit_mapper, noise_model=noise_model)
    sv_mode = False
    num_qubits = 4

else:
    quantum_instance = QuantumInstance(sv_simulator) 
    sv_mode = True
    num_qubits = 4
    
# sweep parameters
all_params = np.arange(-np.pi, np.pi, 0.1)
#all_params = [-0.23]


# what operator mode we should use, do we want to group the paulis, if so what should I record?

def get_h2_hamiltonian(distance=0.75):
    pyscf_driver = PySCFDriver(atom='H .0 .0 {}; H .0 .0 .0'.format(distance),
                               unit=UnitsType.ANGSTROM, charge=0, spin=0, basis='sto3g')
    molecule = pyscf_driver.run()

    core = Hamiltonian(transformation=TransformationType.PH,
                       qubit_mapping=QubitMappingType.JORDAN_WIGNER,
                       two_qubit_reduction=two_qubit_reduction,
                       freeze_core=False,
                       orbital_reduction=[])
    algo_input = core.run(molecule)
    untapered_op = algo_input[0]

    num_orbitals = core.molecule_info['num_orbitals']
    num_particles = core.molecule_info['num_particles']

    return untapered_op,  num_orbitals, num_orbitals, num_particles, num_particles, core


def get_lih_hamiltonian(distance=1.6):
    pyscf_driver = PySCFDriver(atom='Li .0 .0 {}; H .0 .0 .0'.format(distance),
                               unit=UnitsType.ANGSTROM, charge=0, spin=0, basis='sto3g')
    molecule = pyscf_driver.run()

    core = Hamiltonian(transformation=TransformationType.PH,
                       qubit_mapping=QubitMappingType.JORDAN_WIGNER,
                       two_qubit_reduction=two_qubit_reduction,
                       freeze_core=True,
                       orbital_reduction=[])
    algo_input = core.run(molecule)
    untapered_op = algo_input[0]

    num_orbitals_untapered = core.molecule_info['num_orbitals']
    num_particles_untapered = core.molecule_info['num_particles']

    num_qubits = 4
    idx = [1, 5, 7, 11]
    num_orbitals = num_qubits
    num_particles = 2

    h1_new = np.zeros((num_qubits, num_qubits))
    h2_new = np.zeros((num_qubits, num_qubits, num_qubits, num_qubits))

    h1 = molecule.one_body_integrals
    h2 = molecule.two_body_integrals

    for i in range(num_qubits):
        for j in range(num_qubits):
            h1_new[i, j] = h1[idx[i], idx[j]]
            if m == 1:
                continue
            for k in range(num_qubits):
                for l in range(num_qubits):
                    h2_new[i, j, k, l] = h2[idx[i], idx[j], idx[k], idx[l]]

    ferop = FermionicOperator(h1=h1_new, h2=h2_new)
    ferop, energy_shift = ferop.particle_hole_transformation(num_particles)
    
    lih_op = ferop.mapping('jordan_wigner')

    return lih_op, num_orbitals, num_orbitals_untapered, num_particles, num_particles_untapered, core

def evaluate_ground_state(qubit_op, wave_function):


    circuits = qubit_op.construct_evaluation_circuit(statevector_mode = sv_mode, operator_mode='paulis', input_circuit=wave_function)
    result = quantum_instance.execute(circuits)
    avg, std = qubit_op.evaluate_with_result(statevector_mode = sv_mode, result=result)

    return avg, std


def main(molecule, dis=1.6):

    time_in = time.time()

    if molecule == 'lih':
        lih = True
        prefix = 'commutators/lih_1.6'
        the_tapered_op, num_orbitals, num_orbitals_untapered, num_particles, num_particles_untapered, core = get_lih_hamiltonian(distance=dis)
        logger.info("Running LiH @ {}A".format(dis))
    else:
        lih = False
        prefix = ''
        the_tapered_op, num_orbitals, num_orbitals_untapered, num_particles, num_particles_untapered, core  = get_h2_hamiltonian(distance=dis)
        logger.info("Running H2 @ {}A".format(dis))

    if not os.path.exists('commutators'):
        os.makedirs('commutators')
    commutator_prefix = 'commutators/{}_{}'.format(molecule, dis)

    # run the correct sectors only
    init_state = HartreeFock(the_tapered_op.num_qubits, num_orbitals, num_particles,
                             qubit_mapping=qubit_mapping, two_qubit_reduction=two_qubit_reduction)
    var_form = hUCC(the_tapered_op.num_qubits, excitations=[[0, 1, 2, 3]], initial_state=init_state, entanglement='linear')


    se_list_default, de_list_default = UCCSD.compute_excitation_lists(num_particles_untapered, num_orbitals_untapered)

    qr = QuantumRegister(num_qubits,'q')

    log_file = '{}_{}A'.format(molecule, dis)
    outputs = np.empty(len(all_params), dtype=object)
    for i, parameter in enumerate(all_params):

        result = {}
        logger.info("processing {}/{} parameters...".format(i, len(all_params)))
        wave_function = var_form.construct_circuit([parameter], q=qr, qubit_mapper = {0:0,1:1,2:2,3:3})

        energy, std = evaluate_ground_state(the_tapered_op, wave_function) 

        outputs[i] = {}
        outputs[i]['theta']=parameter
        outputs[i]['ground_state_energy']=energy
        outputs[i]['ground_state_std']=std

        eom = QEquationOfMotion(the_tapered_op, num_orbitals_untapered, num_particles_untapered, qubit_mapping=qubit_mapping,
                                    two_qubit_reduction=False, is_eom_matrix_symmetric=True, se_list = se_list_default,
                                    de_list = de_list_default)

        excitation_energies_gap, eom_matrices = eom.calculate_excited_states(wave_function, quantum_instance = quantum_instance,
                                    load_commutators = lih, prefix = prefix)

        outputs[i]['excitation_energies']=excitation_energies_gap

        print(outputs[i])

    np.save('outputs',outputs)
    np.save(log_file,outputs)

    time_out = time.time()
    print('computation time = {}'.format(time_out-time_in))

if __name__ == '__main__':
    # global args
    main('h2', dis = 0.75)

ModuleNotFoundError: No module named 'hucc'